# Basic GAN implementation

### Import relevant modules

In [2]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader
from torch.utils.tensorboard import SummaryWriter
import torchvision.transforms as transforms
import torchvision.datasets as datasets
from torchvision.utils import make_grid
from torch.autograd.variable import Variable

### Check device

In [3]:
device = "cuda" if torch.cuda.is_available() else "cpu"
if device == "cuda":
  print(f"{device} - {torch.cuda.get_device_name()}")
else:
  print(f"{device}")

cuda - GeForce MX130


## 0. Prepare Data

### Image transformations

In [4]:
# Image transformations. They can be chained together using Compose.
transforms = transforms.Compose(
    # Normalize to Mean Standard Deviation
    [transforms.ToTensor(), transforms.Normalize((0.1307,), (0.3081,))]
)

### Define and get dataset

In [5]:
# Image dimensions (flatened)
img_dims = (1, 28, 28)

img_dim = img_dims[0] * img_dims[1] * img_dims[2] # 784

# Defne dataset
dataset = datasets.MNIST(root="dataset/", transform=transforms, download=True)

### Some image functionality

In [6]:
def img2vec(img):
    return img.view(img.size(0), img_dim)

def vec2img(vec):
    return vec.view(vec.size(0), img_dims[0], img_dims[1], img_dims[2])

### Define noise

In [1]:
noise_dim = 100

def noise(size, noise_dim):
    """
    Generates a 1-d vector of gaussian sampled random values
    """
    n = Variable(torch.randn(size, noise_dim))
    return n

## 1. Create Model (Arquitecture)
#### Create Discriminator and Generator classes

In [8]:
class Discriminator(nn.Module):
    """
    A three hidden-layer discriminative neural network
    """
    def __init__(self, img_dim):
        super().__init__()

        self.hidden0 = nn.Sequential(
            nn.Linear(img_dim, 1024),
            # Like RELU but it has a small slope for negative values instead of a flat slope. (In GANs is often a better choice than ReLU)
            nn.LeakyReLU(0.2),
            # To prevent overfitting (see README)
            nn.Dropout(0.3)
        )
        self.hidden1 = nn.Sequential(
            nn.Linear(1024, 512),
            nn.LeakyReLU(0.2),
            nn.Dropout(0.3)
        )
        self.hidden2 = nn.Sequential(
            nn.Linear(512, 256),
            nn.LeakyReLU(0.2),
            nn.Dropout(0.3)
        )
        self.out = nn.Sequential(
            nn.Linear(256, 1),
            nn.Sigmoid()
        )

    def forward(self, x):
        x = self.hidden0(x)
        x = self.hidden1(x)
        x = self.hidden2(x)
        x = self.out(x)
        return x

class Generator(nn.Module):
    """
    A three hidden-layer generative neural network
    """
    # noise_dim is the dimension of the latent noise that the generator takes as input
    def __init__(self, noise_dim, img_dim):
        super().__init__()

        self.hidden0 = nn.Sequential(
            nn.Linear(noise_dim, 256),
            nn.LeakyReLU(0.2)
        )
        self.hidden1 = nn.Sequential(
            nn.Linear(256, 512),
            nn.LeakyReLU(0.2)
        )
        self.hidden2 = nn.Sequential(
            nn.Linear(512, 1024),
            nn.LeakyReLU(0.2)
        )
        self.out = nn.Sequential(
            nn.Linear(1024, img_dim),
            nn.Tanh()
        )

    def forward(self, x):
        x = self.hidden0(x)
        x = self.hidden1(x)
        x = self.hidden2(x)
        x = self.out(x)
        return x

###  Initialize a Discriminator and Generator objects

In [9]:
# Create a Disc object
dis = Discriminator(img_dim).to(device)
# Create a Gen object
gen = Generator(noise_dim, img_dim).to(device)

## 2. Loss and optimizers

### Hyperparameters

In [10]:
lr = 2e-4 # Learning rate
batch_size = 100
num_epochs = 200

### Loss function and optimizers

In [11]:
# Loss function
criterion = nn.BCELoss()

# Optimizers
# Here we tell which parameters (tensors) of the model we should update (dis.parameters(), gen.parameters())
opt_dis = optim.Adam(dis.parameters(), lr=lr)
opt_gen = optim.Adam(gen.parameters(), lr=lr)


### Tensorboard settings

In [12]:
fixed_noise = torch.randn((batch_size, noise_dim)).to(device)
writer_fake = SummaryWriter(f"runs/GAN_MNIST/fake")
writer_real = SummaryWriter(f"runs/GAN_MNIST/real")
step = 0

##  3. Training

In [14]:
### Train Discriminator: max [ log(D(real)) + log(1 - D(G(z)) ]
def train_discriminator(D, optimizer, real, fake):
    # Reset gradients to zero
    D.zero_grad() # Clear out the gradients of all variables
    
    # Train on Real Data
    ## Forward pass
    pred_real = D(real).view(-1)
    ## Loss
    loss_real = criterion(pred_real, torch.ones_like(pred_real))
    ## Backward pass
    loss_real.backward()

    # Train on Fake Data
    ## Forward pass
    pred_fake = D(fake).view(-1)
    ## Loss
    loss_fake = criterion(pred_fake, torch.zeros_like(pred_fake))
    ## Backward pass
    loss_fake.backward()

    # Update weights
    optimizer.step()
    

    return loss_real + loss_fake, pred_real, pred_fake

### Train Generator: min [ log(1 - D(G(z))) ] <-> max [ log(D(G(z))) ]
def train_generator(G, D, optimizer, noise):
    # Reset gradients to zero
    G.zero_grad()

    ## Forward pass
    fake = G(noise)
    ## Loss
    # Calculate fake data: G(noise), evaluate Discriminator prediction D(G(noise)) and compare it to ones_like (we want D to output 1 from G outputs)
    loss = criterion(D(fake), torch.ones_like(D(fake)))
    ## Backward pass
    loss.backward()
    
    # Update weights
    optimizer.step()

    return loss

In [30]:
# Set data
loader = DataLoader(dataset, batch_size=batch_size, shuffle=True)

for epoch in range(num_epochs):
    for batch_idx, (real, _) in enumerate(loader):
        
        
        noisy = noise(batch_size, noise_dim).to(device)

        ## 1. Train Discriminator
        # Real image flatened and sent to device
        real = Variable(img2vec(real)).to(device)
        batch_size = real.shape[0]

        # Generate fake data and detach (so gradients are not calculated for Gen)
        fake = gen(noisy).detach()

        # Train D
        d_loss, d_pred_real, d_pred_fake = train_discriminator(dis, opt_dis, real, fake)

        ## 2. Train Generator
        g_loss = train_generator(gen, dis, opt_gen, noisy)


        # On each epoch at the first mini-batch:
        if batch_idx == 0:
            # Print epochs and losses
            print( 
                f"Epoch [{epoch}/{num_epochs} - "
                f"Loss D: {d_loss:.4f}, Loss G: {g_loss:.4f}]"
            )
            # Get images for each epoch
            with torch.no_grad(): # Context-manager that disable gradient calculation. It will reduce memory consumption.
                fake = vec2img(gen(fixed_noise)) # Fake image with right dimensions
                data = vec2img(real) # Real image with right dimensions
                img_grid_fake = make_grid(fake, normalize=True)
                img_grid_real = make_grid(data, normalize=True)

                writer_fake.add_image(
                    "MNIST Fake Images", img_grid_fake, global_step=step
                )

                writer_real.add_image(
                    "MNIST Real Images", img_grid_real, global_step=step
                )

                step += 1

Epoch [0/200 - Loss D: 1.1602, Loss G: 0.6733]


KeyboardInterrupt: 